# 데이터 셋 내려받기

In [1]:
import tarfile
from pathlib import Path
import urllib.request

def fetch_spam_data():
    spam_root="http://spamassassin.apache.org/old/publiccorpus/"
    ham_url=spam_root+"20030228_easy_ham.tar.bz2"
    spam_url=spam_root+"20030228_spam.tar.bz2"
    
    spam_path=Path()/"datasets"/"spam"
    spam_path.mkdir(parents=True, exist_ok=True)
    for dir_name, tar_name, url in (("easy_ham","ham",ham_url),
                                   ("spam","spam",spam_url)):
        if not (spam_path/dir_name).is_dir():
            path=(spam_path/tar_name).with_suffix(".tar.bz2")
            print("Downloading",path)
            urllib.request.urlretrieve(url, path)
            tar_bz2_file=tarfile.open(path)
            tar_bz2_file.extractall(path=spam_path)
            tar_bz2_file.close()
    return [spam_path/dir_name for dir_name in ("easy_ham","spam")]

In [2]:
ham_dir, spam_dir = fetch_spam_data()

In [3]:
#모든 이메일 읽어들이기
ham_filenames=[f for f in sorted(ham_dir.iterdir()) if len(f.name)>20]
spam_filenames=[f for f in sorted(spam_dir.iterdir()) if len(f.name)>20]

In [4]:
len(ham_filenames)

2500

In [5]:
len(spam_filenames)

500

In [6]:
#list에 PosixPath가 담겨있는 형태
ham_filenames[:5]

[PosixPath('datasets/spam/easy_ham/00001.7c53336b37003a9286aba55d2945844c'),
 PosixPath('datasets/spam/easy_ham/00002.9c4069e25e1ef370c078db7ee85ff9ac'),
 PosixPath('datasets/spam/easy_ham/00003.860e3c3cee1b42ead714c5c874fe25f7'),
 PosixPath('datasets/spam/easy_ham/00004.864220c5b6930b209cc287c361c99af1'),
 PosixPath('datasets/spam/easy_ham/00005.bf27cdeaf0b8c4647ecd61b1d09da613')]

In [7]:
#모듈을 사용해 이메일 파싱하기
import email
import email.policy

def load_email(filepath):
    with open(filepath, "rb") as f:
        return email.parser.BytesParser(policy=email.policy.default).parse(f)

In [8]:
ham_emails=[load_email(filepath) for filepath in ham_filenames]
spam_emails=[load_email(filepath) for filepath in spam_filenames]

In [9]:
#list에 각 이메일이 저장되어 있음
ham_emails[:5]

In [10]:
#email 내용 오픈을 위해선 get_contnt 함수 사용 필수
print(ham_emails[1].get_content().strip())

Martin A posted:
Tassos Papadopoulos, the Greek sculptor behind the plan, judged that the
 limestone of Mount Kerdylio, 70 miles east of Salonika and not far from the
 Mount Athos monastic community, was ideal for the patriotic sculpture. 
 
 As well as Alexander's granite features, 240 ft high and 170 ft wide, a
 museum, a restored amphitheatre and car park for admiring crowds are
planned
---------------------
So is this mountain limestone or granite?
If it's limestone, it'll weather pretty fast.

------------------------ Yahoo! Groups Sponsor ---------------------~-->
4 DVDs Free +s&p Join Now
http://us.click.yahoo.com/pt6YBB/NXiEAA/mG3HAA/7gSolB/TM
---------------------------------------------------------------------~->

To unsubscribe from this group, send an email to:
forteana-unsubscribe@egroups.com

 

Your use of Yahoo! Groups is subject to http://docs.yahoo.com/info/terms/


어떤 이메일은 이미지나 첨부파일을 가진 멀티파트로 구성

In [54]:
def get_email_structure(email):
    if isinstance(email, str):
        return email
    payload=email.get_payload()
    if isinstance(payload, list):
        multipart=", ".join([get_email_structure(sub_email)
                           for sub_email in payload])
        return f"multipart({multipart})"
    else:
        return email.get_content_type()

In [55]:
from collections import Counter

def structures_counter(emails):
    structures=Counter()
    for email in emails:
        structure=get_email_structure(email)
        structures[structure]+=1
    return structures

In [56]:
structures_counter(ham_emails).most_common()

[('text/plain', 2408),
 ('multipart(text/plain, application/pgp-signature)', 66),
 ('multipart(text/plain, text/html)', 8),
 ('multipart(text/plain, text/plain)', 4),
 ('multipart(text/plain)', 3),
 ('multipart(text/plain, application/octet-stream)', 2),
 ('multipart(text/plain, text/enriched)', 1),
 ('multipart(text/plain, application/ms-tnef, text/plain)', 1),
 ('multipart(multipart(text/plain, text/plain, text/plain), application/pgp-signature)',
  1),
 ('multipart(text/plain, video/mng)', 1),
 ('multipart(text/plain, multipart(text/plain))', 1),
 ('multipart(text/plain, application/x-pkcs7-signature)', 1),
 ('multipart(text/plain, multipart(text/plain, text/plain), text/rfc822-headers)',
  1),
 ('multipart(text/plain, multipart(text/plain, text/plain), multipart(multipart(text/plain, application/x-pkcs7-signature)))',
  1),
 ('multipart(text/plain, application/x-java-applet)', 1)]

In [57]:
structures_counter(spam_emails).most_common()

[('text/plain', 218),
 ('text/html', 183),
 ('multipart(text/plain, text/html)', 45),
 ('multipart(text/html)', 20),
 ('multipart(text/plain)', 19),
 ('multipart(multipart(text/html))', 5),
 ('multipart(text/plain, image/jpeg)', 3),
 ('multipart(text/html, application/octet-stream)', 2),
 ('multipart(text/plain, application/octet-stream)', 1),
 ('multipart(text/html, text/plain)', 1),
 ('multipart(multipart(text/html), application/octet-stream, image/jpeg)', 1),
 ('multipart(multipart(text/plain, text/html), image/gif)', 1),
 ('multipart/alternative', 1)]

각각 구성 내용을 살펴보자면 햄 이메일들은 대부분 plain text이고 스팸 메일들은 html이 차지하는 비중이 큼

햄 이메일들은 서명을 포함하는 경우가 종종 있음

In [58]:
#이메일 헤더 살펴보기
for header, value in spam_emails[0].items():
    print(header, ":", value)

Return-Path : <12a1mailbot1@web.de>
Delivered-To : zzzz@localhost.spamassassin.taint.org
Received : from localhost (localhost [127.0.0.1])	by phobos.labs.spamassassin.taint.org (Postfix) with ESMTP id 136B943C32	for <zzzz@localhost>; Thu, 22 Aug 2002 08:17:21 -0400 (EDT)
Received : from mail.webnote.net [193.120.211.219]	by localhost with POP3 (fetchmail-5.9.0)	for zzzz@localhost (single-drop); Thu, 22 Aug 2002 13:17:21 +0100 (IST)
Received : from dd_it7 ([210.97.77.167])	by webnote.net (8.9.3/8.9.3) with ESMTP id NAA04623	for <zzzz@spamassassin.taint.org>; Thu, 22 Aug 2002 13:09:41 +0100
From : 12a1mailbot1@web.de
Received : from r-smtp.korea.com - 203.122.2.197 by dd_it7  with Microsoft SMTPSVC(5.5.1775.675.6);	 Sat, 24 Aug 2002 09:42:10 +0900
To : dcek1a1@netsgo.com
Subject : Life Insurance - Why Pay More?
Date : Wed, 21 Aug 2002 20:31:57 -1600
MIME-Version : 1.0
Message-ID : <0103c1042001882DD_IT7@dd_it7>
Content-Type : text/html; charset="iso-8859-1"
Content-Transfer-Encoding : qu

In [59]:
spam_emails[0]["Subject"]

'Life Insurance - Why Pay More?'

# 훈련 세트와 테스트 세트로 분리하기

In [60]:
import pandas as pd
import numpy as np

data_set=np.array(ham_emails+spam_emails,dtype=object)
data_label=np.array([0]*len(ham_emails)+[1]*len(spam_emails))

In [61]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(data_set, data_label,test_size=0.1,random_state=42)

전처리 함수 작성: HTML을 일반 텍스트로 변환하는 함수 
- head 섹션을 삭제하고 모든 a 태그를 HYPERLINK 문자로 바꿈
- 모든 HTML 태그를 제거하고 텍스트만 남김
- 보기 편하게 여러개의 개행 문자를 하나로 만들고 html 엔티티를 복원

# 변환 파이프라인 작성

In [73]:
import re
from html import unescape

def html_to_plain_text(html):
    text=re.sub('<head.*?>.*?</head>', '', html, flags=re.M|re.S|re.I)
    text=re.sub('<a\s.*?>',' HYPERLINK ', text, flags=re.M|re.S|re.I)
    text=re.sub('<.*?>', '', text, flags=re.M|re.S)
    text=re.sub(r'(\s*\n)+', '\n', text, flags=re.M|re.S)
    return unescape(text)

In [74]:
html_spam_emails=[email for email in X_train[y_train==1]
                 if get_email_structure(email)=="text/html"]

In [75]:
sample_html_spam=html_spam_emails[1]
print(sample_html_spam.get_content().strip()[:1000],"...")

<html>

<head>
<meta http-equiv="Content-Language" content="en-us">
<meta name="GENERATOR" content="Microsoft FrontPage 5.0">
<meta name="ProgId" content="FrontPage.Editor.Document">
<meta http-equiv="Content-Type" content="text/html; charset=windows-1252">
<title>Norton AD</title>
</head>

<body>

<table border="3" cellspacing="3" width="469" cellpadding="3" bgcolor="#000080"  bordercolor="#FFFFFF" style="border-collapse: collapse" bordercolordark="#FFFFFF">
  <tr>
    <td width="447">&nbsp;
      <table border="1" width="100%" bgcolor="#FFFFFF" bordercolor="#000080">
        <tr>
          <td width="100%" align="center"><font face="Impact" size="4" color="#CC0000">ATTENTION:
            This is a MUST for <u>ALL</u> Computer Users!!!</font></td>
        </tr>
      </table>
      <p align="center"><font size="3" face="Verdana"><b>&nbsp;<font color="#FFFFFF">*NEW
      - Special Package Deal!*</font></b></font></p>
      <table border="8" width="100%" height="86" bgcolor="#FFFFFF" bo

In [67]:
print(html_to_plain_text(sample_html_spam.get_content())[:1000],"...")


     
          ATTENTION:
            This is a MUST for ALL Computer Users!!!
       *NEW
      - Special Package Deal!*
          Norton
            SystemWorks 2002 Software Suite
            -Professional Edition-
          Includes
            Six - Yes 6! - Feature-Packed UtilitiesALL
            for 1
            Special LOW
            Price!
          This Software Will:- Protect your
            computer from unwanted and  hazardous viruses- Help secure your
            private & valuable information- Allow you to transfer files
            and send e-mails  safely- Backup your ALL your data quick and
            easily- Improve your PC's performance w/superior
            integral diagnostics!
       
            6
            Feature-Packed Utilities
            1
             Great
            Price
            A $300+   Combined Retail Value
            YOURS for Only $29.99!
            <Includes
            FREE Shipping!>
     Don't fall prey to destructive viruses
 

In [76]:
def email_to_text(email):
    html=None
    
    #walk: 메세지 객체 트리의 모든 파트와 서브 파트를 이터레이트하는데 사용되는 범용 제너레이터
    for part in email.walk(): 
        ctype=part.get_content_type()
        if not ctype in ("text/plain","text/html"):
            continue
        try:
            content=part.get_contnet()
        except: #encoding issue 발생시
            content=str(part.get_payload())
        if ctype=="text/plain":
            return content
        else:
            html=content
    if html:
        return html_to_plain_text(html)

In [77]:
print(email_to_text(sample_html_spam)[:100],"...")


     
          ATTENTION:
            This is a MUST for ALL Computer Users!!!
       *NEW
      - ...


In [70]:
#자연어 처리 툴킷 NLTK를 이용해 어간 추출
import nltk

stemmer=nltk.PorterStemmer()
for word in ("Computations","Computation","Computing","Computed","Compute",
            "Compulsive"):
    print(word, "=>", stemmer.stem(word))

Computations => comput
Computation => comput
Computing => comput
Computed => comput
Compute => comput
Compulsive => compuls


In [28]:
%pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [29]:
%pip install -q -U urlextract

Note: you may need to restart the kernel to use updated packages.


In [45]:
#urlextract 라이브러리를 사용해 인터넷 주소를 "URL" 문자로 바꾸기
import urlextract

url_extractor=urlextract.URLExtract()
some_text="Will it detect github.com and https://youtu.be"
print(url_extractor.find_urls(some_text))

['github.com', 'https://youtu.be']


In [78]:
#위 작업을 하나의 변환기로 연결하여 이메일을 단어 카운트로 바꾸기
from sklearn.base import BaseEstimator, TransformerMixin

class EmailToWordCounterTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self, strip_headers=True, lower_case=True,
                remove_punctuation=True, replace_urls=True,
                replace_numbers=True, stemming=True):
        self.strip_headers=strip_headers
        self.lower_case=lower_case
        self.remove_punctuation=remove_punctuation
        self.replace_urls=replace_urls
        self.replace_numbers=replace_numbers
        self.stemming=stemming

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_transformed=[]
        for email in X:
            text=email_to_text(email) or ""
            if self.lower_case:
                text=text.lower()
            if self.replace_urls and url_extractor is not None:
                urls=list(set(url_extractor.find_urls(text)))
                urls.sort(key=lambda url:len(url), reverse=True)
                for url in urls:
                    text=text.replace(url," URL ")
            if self.replace_numbers:
                text=re.sub(r'\d+(?:\.\d*)?(?:[eE][+-]?\d+)?','NUMBER',text)
            if self.remove_punctuation:
                text=re.sub(r'\W+',' ', text,flags=re.M)
            word_counts=Counter(text.split())
            if self.stemming and stemmer is not None:
                stemmed_word_counts=Counter()
                for word,count in word_counts.items():
                    stemmed_word=stemmer.stem(word)
                    stemmed_word_counts[stemmed_word]+=count
                word_counts=stemmed_word_counts
            X_transformed.append(word_counts)
        return np.array(X_transformed)

In [79]:
X_few=X_train[:3]
X_few_wordcounts=EmailToWordCounterTransformer().fit_transform(X_few)
X_few_wordcounts

array([Counter({'the': 19, 'cd': 8, 'it': 8, 'would': 7, 'player': 6, 'to': 5, 'start': 5, 'a': 5, 'and': 5, 'be': 5, 'play': 4, 'toddler': 3, 'immedi': 3, 'like': 3, 'insert': 3, 'is': 3, 'at': 3, 'you': 3, 'write': 2, 'for': 2, 'press': 2, 'music': 2, 'or': 2, 'some': 2, 'that': 2, 'after': 2, 'd': 2, 'an': 2, 'hmm': 2, 'rip': 2, 'cach': 2, 'face': 2, 'seek': 2, 'of': 2, 'recogn': 2, 'which': 2, 'from': 2, 'mr': 1, 'fork': 1, 'jim': 1, 'whitehead': 1, 'must': 1, 'caus': 1, 'within': 1, 'half': 1, 'second': 1, 'get': 1, 'causal': 1, 'not': 1, 'button': 1, 'multipl': 1, 'time': 1, 'sound': 1, 'indic': 1, 'will': 1, 'real': 1, 'soon': 1, 'now': 1, 'tonal': 1, 'countdown': 1, 'nice': 1, 'what': 1, 'ideal': 1, 'wa': 1, 'load': 1, 'mpnumber': 1, 'with': 1, 'solid': 1, 'state': 1, 'storag': 1, 'instant': 1, 'on': 1, 'seem': 1, 'everi': 1, 'should': 1, 'includ': 1, 'capabl': 1, 'encod': 1, 'last': 1, 'few': 1, 'playback': 1, 'then': 1, 'never': 1, 'need': 1, 'delay': 1, 'initi': 1, 'onli': 1

단어 카운트를 벡터로 변환하는 변환기 작성하기

In [82]:
from scipy.sparse import csr_matrix

class WordCounterToVectorTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self, vocabulary_size=1000):
        self.vocabulary_size=vocabulary_size
        
    #자주 나타나는 단어 순으로 정렬된 언어 목록 생성
    def fit(self, X, y=None):
        total_count=Counter()
        for word_count in X:
            for word, count in word_count.items():
                total_count[word]+=min(count,10)
        most_common=total_count.most_common()[:self.vocabulary_size]
        self.vocabulary_={word: index+1
                         for index, (word,count) in enumerate(most_common)}
        return self
    
    def transform(self, X, y=None):
        rows=[]
        cols=[]
        data=[]
        for row, word_count in enumerate(X):
            for word, count in word_count.items():
                rows.append(row)
                cols.append(self.vocabulary_.get(word, 0))
                data.append(count)
        return csr_matrix((data,(rows, cols)),
                         shape=(len(X),self.vocabulary_size+1))

In [84]:
vocab_transformer=WordCounterToVectorTransformer(vocabulary_size=10)
X_few_vectors=vocab_transformer.fit_transform(X_few_wordcounts)
X_few_vectors

<3x11 sparse matrix of type '<class 'numpy.int64'>'
	with 31 stored elements in Compressed Sparse Row format>

In [85]:
X_few_vectors.toarray()

array([[172,  19,   5,   8,   5,   5,   0,   1,   2,   3,   5],
       [167,  10,   6,   5,   4,   4,  10,   7,   1,   0,   1],
       [492,  54,  17,   8,  10,  15,   9,   7,  11,  10,   6]])

In [87]:
vocab_transformer.vocabulary_

{'the': 1,
 'to': 2,
 'it': 3,
 'a': 4,
 'and': 5,
 'number': 6,
 'in': 7,
 'of': 8,
 'is': 9,
 'be': 10}

# 전체 데이터셋 변환

In [88]:
from sklearn.pipeline import Pipeline

preprocess_pipeline=Pipeline([
    ("email_to_wordcount", EmailToWordCounterTransformer()),
    ("wordcount_to_vector", WordCounterToVectorTransformer()),
])

X_train_transformed=preprocess_pipeline.fit_transform(X_train)

# 모델 선택과 훈련

In [89]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

log_clf=LogisticRegression(max_iter=1000, random_state=42)
score=cross_val_score(log_clf, X_train_transformed, y_train, cv=3)

In [91]:
#정확도 
score.mean()

0.987037037037037

In [106]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import make_pipeline

tree_reg=make_pipeline(preprocess_pipeline, DecisionTreeRegressor(random_state=42))
tree_reg.fit(X_train,y_train)

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('email_to_wordcount',
                                  EmailToWordCounterTransformer()),
                                 ('wordcount_to_vector',
                                  WordCounterToVectorTransformer())])),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(random_state=42))])

In [108]:
from sklearn.metrics import root_mean_squared_error

tree_pred=tree_reg.predict(X_train)
tree_rmse=root_mean_squared_error(y_train, tree_pred)

tree_rmse

0.0

# 정확도 평가하기

In [103]:
from sklearn.metrics import precision_score, recall_score

X_test_transformed=preprocess_pipeline.transform(X_test)

log_clf=LogisticRegression(max_iter=1000,random_state=42)
log_clf.fit(X_train_transformed, y_train)

y_pred=log_clf.predict(X_test_transformed)

In [104]:
print(f"Precision: {precision_score(y_test, y_pred):.2%}")
print(f"Recall: {recall_score(y_test, y_pred):.2%}")

Precision: 96.08%
Recall: 98.00%
